In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
from multiprocessing import Pool
import time,gc

In [2]:
# 上周同期同时刻进站
def construct_df():
    final = pd.DataFrame()
    for i in range(81):
        temp = pd.DataFrame()
        temp['minute'] = list(range(1440))
        temp['stationID'] = i
        final = final.append(temp)
    final['minute'] = final['minute']/10
    return final
# 统计进出站每天的变化情况
def get_daily_activity(i):
    if i<10:
        s = '0'+str(i)
    else:
        s = str(i)
    day_activ = pd.read_csv('data/Metro_train/record_2019-01-%s.csv'%s)
    day_activ['time'] = pd.to_datetime(day_activ.time)
    day_activ['stationID'] = day_activ['stationID']
    day_activ['deviceID'] = day_activ['deviceID']
    day_activ['status'] = day_activ['status']
    day_activ['payType'] = day_activ['payType']
    day_activ['lineID'] = day_activ['lineID'].apply(lambda x: {'A':0,'B':2,'C':1}[x])
    return day_activ

In [3]:
all_df = pd.DataFrame()
#for date in range(7,26):
for date in tqdm(list(range(1,26))+[28]):
    final = construct_df()
    day_activ = get_daily_activity(date)[['time','lineID','stationID','deviceID','status','payType']]
    df = pd.DataFrame()
    for i in range(10):
        day_activ['minute'] = (day_activ['time'].dt.minute-i+day_activ['time'].dt.hour*60)//10+0.1*i
        day_activ['minute'] = day_activ['minute'].apply(lambda x:143+1+x if x<0 else x)
        df = df.append(day_activ)
    df.sort_values(by='minute',inplace=True)
    come = df[df.status==1]
    come_group = come.groupby(['stationID','minute'],as_index=False).agg({'status':'count'})
    final = final.merge(come_group,on=['stationID','minute'],how='left')
    final['date'] =date
    all_df = all_df.append(final)


100%|██████████| 26/26 [07:06<00:00, 17.35s/it]


In [4]:
all_df.fillna(0,inplace=True)
all_df['last_week'] = [0]*(final.shape[0]*8)+list(all_df['status'])[final.shape[0]*8:]

In [5]:
import re
re.findall(r"\d+\.?\d*",'91良')

['91']

In [6]:
weather = pd.read_csv('data/weather.csv',header=None).rename(columns={1:'max_t',2:'min_t',5:'air'})
weather['max_t'] = weather['max_t'].apply(lambda x:x.split('℃')[0])
weather['min_t'] = weather['min_t'].apply(lambda x:x.split('℃')[0])
weather['air'] = weather['air'].apply(lambda x:re.findall(r"\d+\.?\d*",x)[0])
weather['weather'] = weather[3].apply(lambda x:len(x))
weather = weather.reset_index().rename(columns={'index':'date'})

In [8]:
weather.head()

,date,0,max_t,min_t,3,4,air,weather
0,0,2019-01-01星期二,5,2,多云~小雨,西北风2级,76,5
1,1,2019-01-02星期三,6,3,小雨~阴,东北风1级,91,4
2,2,2019-01-03星期四,8,5,阴~小雨,东北风1级,128,4
3,3,2019-01-04星期五,7,4,中雨,西北风1级,87,2
4,4,2019-01-05星期六,8,4,小雨~阴,西北风2级,34,4


In [10]:
weather[['date','max_t','min_t','air','weather']].to_pickle('features/weather.pkl')